# Taschenrechner Revisited

In Woche 3 wurde ein sehr einfacher Taschenrechner implementiert, welcher jetzt erweitert werden soll mit Error Handling.

Implementieren Sie eine Funktion `calculate`, welche drei Parameter entgegennimmt:
- `number1`: Eine Zahl
- `number2`: Eine Zahl
- `operator`: Ein String, welcher eine der vier Grundrechenarten enthält: `+`, `-`, `*` oder `/`

Falls `operator` nicht einer der vier Grundrechenarten entspricht, soll eine Exception vom Typ `ValueError` geworfen werden.

In [2]:
def calculate(number1, number2, operator):
    if operator == '+':
        return number1 + number2
    elif operator == '-':
        return number1 - number2
    elif operator == '*':
        return number1 * number2
    elif operator == '/':
        return number1 / number2
    else:
        raise ValueError("Operator muss +, -, * oder / sein")

Teste die Funktion mit folgenden Aufrufen:

In [6]:
print(calculate(1, 2, '+'))

3


In [7]:
print(calculate(1, 2, '-'))

-1


In [8]:
print(calculate(1, 2, 'mal'))

ValueError: Operator muss +, -, * oder / sein

Wenn Zahlen eingelesen von einem Benutzer eingelesen werden können auch schon Probleme auftreten. Zum Beispiel wenn der Benutzer eine Zeichenkette eingibt, welche nicht in eine Zahl umgewandelt werden kann. In diesem Fall wird eine Exception vom Typ `ValueError` geworfen.

Entwickle eine Funktion `ask_for_number` mit folgender Funktionalität:
- Die Funktion fragt Anwender:innen nach einer Zahl 
- Der Benutzer wird solange aufgefordert, eine neue Zahl einzugeben bis keine Error mehr auftritt
- Diese Zahl wird zurückgegeben

In [5]:
def ask_for_number():
    while True:
        try:
            answer = input("Bitte Zahl eingeben: ")
            number = float(answer)
            return number
        except ValueError:
            print("Das war keine Zahl. Bitte nochmal versuchen.")


Jetzt wollen wir das ganze in eine kleine Applikation einbetten, mit welcher die Inputs vom Benutzer entgegengenommen werden können.

Schreibe eine Funktion `calculator` welche folgende Aufgaben erfüllt:
- Fragt den Benutzer nach einer Zahl mit `ask_for_number`
- Fragt den Benutzer nach einer weiteren Zahl mit `ask_for_number`
- Fragt den Benutzer nach einer der vier Grundrechenarten
- Ruft die Funktion `calculate` auf und gibt das Resultat aus
- Behandelt die Exceptions, welche von `calculate` geworfen werden können

In [9]:
def calculator():
    number1 = ask_for_number()
    number2 = ask_for_number()
    operator = input("Bitte geben Sie den Operator ein: ")
    try:
        print(calculate(number1, number2, operator))
    except ValueError as e:
        print("Operator muss +, -, * oder / sein")
    except ZeroDivisionError as e:
        print("Division durch 0 ist nicht erlaubt")

In [11]:
calculator()

Division durch 0 ist nicht erlaubt


Versuche eigenständig noch die `ZeroDivisionError` zu behandeln in der Funktion `calculator`.

# 2. ID-Nummer Parsen

Auf der Schweizer ID gibt es maschinenlesbare Zeilen, welche die wichtigsten Informationen enthalten. In dieser Aufgabe schreiben wir ein Programm, welche diese Information extrahiert.

Die maschinenlesbaren Zeilen sehen wie folgt aus:
```
IDCHE123412345<7<<<<<<<<<<<<<<
0402291M1001015CHE<<<<<<<<<<<8
MUSTERMANN<<MAX<OTTO<<<<<<<<<<
```

Hier als Definition:

In [1]:
beispiel = """
IDCHE123412345<7<<<<<<<<<<<<<<
0402291M1001015CHE<<<<<<<<<<<8
MUSTERMANN<<MAX<OTTO<<<<<<<<<<
"""

Beim Einlesen wollen wir alle Leerschläge und Zeilenumbrüche entfernen. Schreibe eine Funktion `whitespace_entfernen`, welches für eine beliebige `str` alle Leerschläge und Zeilenumbrüche entfernt und die bereinigte `str` zurückgibt.

In [2]:
# Lösung
def whitespace_entfernen(string):
    return string.replace("\n", "").replace(" ", "")

Wir verwenden die Funktion wie folgt:

In [3]:
beispiel_ohne_whitespace = whitespace_entfernen(beispiel)
beispiel_ohne_whitespace

'IDCHE123412345<7<<<<<<<<<<<<<<0402291M1001015CHE<<<<<<<<<<<8MUSTERMANN<<MAX<OTTO<<<<<<<<<<'

Nun können wir verschiedene Informationen extrahieren. 
Nach `IDCHE` ist die ID-Nummer enthalten (im Beispiel 123412345). Schreibe eine Funktion `id_nummer`, welche aus einer bereinigten `str` die ID-Nummer extrahiert.

In [36]:
# Lösung
def id_nummer(ohne_whitespace):
    return ohne_whitespace[6:14]

Der Name ist auf der dritten Zeile (und beginnt daher bei Zeichen 60). Zuerst kommt der Nachname, dann zwei `<<`, dann ein oder mehrere Vornamen getrennt durch nur ein `<`. Schreibe eine Funktion `name_extrahieren`, welche den Namen in folgendem Format zurückgibt:

`Nachname, Vorname1 Vorname2`

In unserem Fall: `Mustermann, Max Otto`

Tipp: Um jedes Wort mit einem Grossbuchstaben zu beginnen, haben `str`-Objekte in Python die Methode `.title()`.

In [37]:
# Lösung
def name_extrahieren(ohne_whitespace):
    letzte_zeile = ohne_whitespace[60:]
    ohne_zeichen_am_ende = letzte_zeile.strip("<")
    mit_luecken_und_komma = ohne_zeichen_am_ende.replace("<<", ", ").replace("<", " ")
    erster_buchstaben_gross = mit_luecken_und_komma.title()
    return erster_buchstaben_gross

Das Geburtsdatum ist auf der zweiten Zeile, im Format JJMMDD (zwei Stellen für das Jahr, zwei für den Monat, zwei für den Tag). In unserem Beispiel ist der relevante String `040229`, die Person ist also am 29. Februar 04 geboren. Schreibe eine Funktion `geburtsdatum_extrahieren`, welche das Datum extrahiert und wie folgt zurückgibt: `29. 2. 2004`. Um festzustellen, ob es `20..` oder `19..` ist, überprüfe ob das Geburtsjahr kleiner oder gleich `23` ist. 

In [38]:
def geburtsdatum_extrahieren(ohne_whitespace):
    datum = ohne_whitespace[30:36]

    tag = int(datum[:2])
    monat = int(datum[2:4])
    jahr = int(datum[4:6])

    if jahr <= 23:
        jahr_vollstaendig = 2000 + jahr
    else:
        jahr_vollstaendig = 1900 + jahr

    # wir rechnen tag und monat zuerst nach int und dann wieder nach str, um das vorangehende 0 zu entfernen
    return str(tag) + ". " + str(monat) + ". " + str(jahr_vollstaendig)

In [39]:
geburtsdatum_extrahieren(beispiel_ohne_whitespace)

'4. 2. 1929'

Packe nun das ganze in eine grosse Funktion `id_einlesen` zusammen, welche nach den maschinenlesbaren Zeilen fragt und dann den Namen, das Geburtsdatum sowie die ID-Nummer mit `print` ausgibt.

In [40]:
# Lösung
def id_einlesen(id_zeilen):
    ohne_whitespace = whitespace_entfernen(id_zeilen)
    print("Name:", name_extrahieren(ohne_whitespace))
    print("Geburtstag:", geburtsdatum_extrahieren(ohne_whitespace))
    print("ID-Nummer:", id_nummer(id_zeilen))

In [41]:
id_einlesen(beispiel)

Name: Mustermann, Max Otto
Geburtstag: 4. 2. 1929
ID-Nummer: 12341234


Bonusaufgabe: 

Schreibe ein Programm, welches auf Grund der ID-Zeilen ausgibt, ob die Person bereits 16 Jahre alt ist - um beispielsweise an einem Zigarettenautomat zu überprüfen, ob Kunden bereits genügend alt sind.